# Web Scraping 

In [1]:
from youtube_comment_scraper_python import *
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.common.by import By
import requests
import json

from warnings import filterwarnings
filterwarnings('ignore')

Starting DataKund...


# Links y Names

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.youtube.com/c/DelcaVideography/videos')

sleep(5)

height = driver.execute_script("return document.documentElement.scrollHeight")
previousHeight = -1

while previousHeight < height:
    previousHeight = height
    driver.execute_script(f'window.scrollTo(0,{height + 10000})')
    sleep(1)
    height = driver.execute_script("return document.documentElement.scrollHeight")

# Lista de Links
page_links = [element.get_attribute("href") for element in driver.find_elements("xpath",'//*[@id="video-title"]')]

# info videos
names_info =  [element.get_attribute("aria-label") for element in driver.find_elements("xpath",'//*[@id="video-title"]')]

# Duración video
duracion_video =  [element.get_attribute("aria-label") for element in driver.find_elements("xpath",'//*[@id="text"]')]

driver.close()

#---------------------------------------------------------------------------------------------------------------------------------------

# Carpinteria

#Quitando los None de la lista
names_info = [i for i in names_info if i is not None]

duracion_video = [i for i in duracion_video if i is not None]

#info_videos = [names_info[i][names_info[i].find('hace'):] for i in range(len(names_info))]   # Había elementos con hace en su titulo, tocó ajustar
info_videos = [names_info[i].split('hace')[-1].split('vistas')[0] for i in range(len(names_info))]
info_videos = [i + 'visitas' for i in info_videos]

# Nombres de vios hasta hace del string
names_videos = [names_info[i][:[names_info[i].find('hace ')][0]] for i in range(len(names_info))]   

# Enlaces para sacar el ID

page_links_listos = []

for i in page_links:

    if 'short' in i:
        enlace = i.split('/')[-1]
        page_links_listos.append(enlace)

    else:
        enlace = i.split('=')[-1]
        page_links_listos.append(enlace)

# df con la info
data_videos = pd.DataFrame()

data_videos['names_videos'] = names_videos
data_videos["names_videos"] = data_videos["names_videos"].str.strip() 

data_videos['info_videos'] = info_videos
data_videos["info_videos"] = data_videos["info_videos"].str.strip() 

data_videos['duracion_video'] = duracion_video
data_videos["duracion_video"] = data_videos["duracion_video"].str.strip() 

data_videos['page_links'] = page_links

data_videos['id_videos'] = page_links_listos

data_videos

,names_videos,info_videos,duracion_video,page_links,id_videos
0,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...,hace 5 días 8 minutos y 50 segundos 784 vistas,8 minutos y 50 segundos,https://www.youtube.com/watch?v=cZUmPLANTy0,cZUmPLANTy0
1,💚 TODO sobre la herramienta CURVA en CorelDRAW...,"hace 2 semanas 14 minutos y 11 segundos 2,127 ...",14 minutos y 11 segundos,https://www.youtube.com/watch?v=qZDnqNKSp1U,qZDnqNKSp1U
2,🏃🏼‍♂️ Speedrun LOGO CANGREJO #shorts ▶ DelcaVi...,hace 3 semanas 1 minuto y 5 segundos 368 vistas,1 minuto y 5 segundos,https://www.youtube.com/watch?v=q3FV1NWUVG0,q3FV1NWUVG0
3,⭐ Haz tus DISEÑOS con PROPORCIÓN ÁUERA ▶ Tutor...,"hace 3 semanas 10 minutos y 55 segundos 1,028 ...",10 minutos y 55 segundos,https://www.youtube.com/watch?v=EnxrHicdECg,EnxrHicdECg
4,💚 La herramienta ZOOM Explicada ▶ TUTORIAL Cor...,hace 1 mes 7 minutos y 24 segundos 759 vistas,7 minutos y 24 segundos,https://www.youtube.com/watch?v=tF5T5zm2Kk4,tF5T5zm2Kk4
...,...,...,...,...,...
528,El baño del gato de DelcaVideography,"hace 14 años 7 minutos y 6 segundos 2,563 vistas",7 minutos y 6 segundos,https://www.youtube.com/watch?v=TQFNi7CAb2Q,TQFNi7CAb2Q
529,"Bancolombia, Medellín, Building, rascasuelos, ...","hace 14 años 1 minuto y 15 segundos 2,850 vistas",1 minuto y 15 segundos,https://www.youtube.com/watch?v=9DzUGseMs2c,9DzUGseMs2c
530,Edificio Bancolombia Medellín de DelcaVideography,"hace 14 años 28 segundos 11,647 vistas",28 segundos,https://www.youtube.com/watch?v=ORDXcEX7xi8,ORDXcEX7xi8
531,DUCON sistemas modulares para oficina de Delca...,"hace 14 años 5 minutos y 2 segundos 2,055 vistas",5 minutos y 2 segundos,https://www.youtube.com/watch?v=2fxhnwxRmN4,2fxhnwxRmN4


# Likes

In [8]:
lista_df_lik = []

ID_videos = data_videos.id_videos

for i in ID_videos:

    url = "https://returnyoutubedislikeapi.com/votes"

    querystring = {"videoId":i,"likeCount":""}

    payload = ""
    response = requests.request("GET", url, data=payload, params=querystring)

    info_id = json.loads(response.text)

    df = pd.DataFrame()
    df = df.append(info_id, ignore_index=True)
    lista_df_lik.append(df)
    sleep(0.5)

    print(i)

data_likes = pd.concat(lista_df_lik)
data_likes.drop([i for i in ['type', 'title', 'status','traceId', 'deleted'] if i in data_likes.columns], axis=1, inplace = True)

data_likes

cZUmPLANTy0
qZDnqNKSp1U
q3FV1NWUVG0
EnxrHicdECg
tF5T5zm2Kk4
8NdzbkCYcwo
FH_mrkzDz5g
vNWxJoxOHFA
IyIvsSRNGZk
t95kXSMyNYo
Fr9VZumHykY
GxEo1VnicN4
9SKCAwbvsFI
lbkHP_KKD5s
5lzcNprr2R8
d4W0C_QPDiE
xK5DzITQgMM
EetpcWHa3QM
b9_XfrLRhbk
jevAvwR4Pl8
yS4dxKwYzY8
UX7utBiADpI
zdRd8ybbfXU
RDLqK5qDRco
eTNr9599FKg
6my2PObwnlY
kNMd8vuFhXs
hKrL3UaIcN8
fQlojG2yhPk
4AqdI-t6S9c
4QfQjGVMUqA
vyg5_ZPlMfA
YL0lHOKtXT4
GvWl6xRiLlA
SChyIBkkCfg
alDiKvBy7VQ
qYt3TLlMkOU
qb1g0l0s-8M
O1Z08LHtUto
SfJvprvv2SY
uKd41FnPlhU
tRzGALmKrxY
wIU7eXRdj48
XrQZ3jFi738
ghc_2Ljua5A
_EjDfu5upcg
jPTShYN3wYo
1WflrST7JBI
GvsRcuPy0bs
DTghfyfXNoQ
34oSrvGeogo
9TqoaxoutJA
9OONS6UnE8E
zCs2HPG-wQU
3Zai99qd9kQ
IGZB-kIGE6I
vg8pGU09qQ4
C9jBrGCi2LE
X8e-3u0kN_E
E_i_qR73Q9g
0slsGhZDC1Q
uQ8jd0fvQug
FhNdnP77p4M
FYS3pPudW10
rQKwYTmFz7I
dfJzsYz4QeA
EncVwqu2W6E
DpuEqUy3IsQ
wKQdpbEgCQk
tJh3qoj5uL0
FC8Ys6hqa2c
DWmNpU8Amco
kuI3uJebbgk
JhBKvI8Dcnk
5WcmcnMkBHw
sOYygjhplfw
6yA8gjsJws4
q9Um7q-MN28
4oAC_IUKdss
OO2CDzNmVB0
vCMrop1rNp0
jRBz2XK3W-U
UzS9Iar4trA
dD9p

,id,dateCreated,likes,dislikes,rating,viewCount
0,cZUmPLANTy0,2022-10-30T23:52:23.41786Z,63,0,5.000000,784
0,qZDnqNKSp1U,2022-10-30T20:30:47.357809Z,144,1,4.972414,2117
0,q3FV1NWUVG0,2022-10-08T16:50:03.740318Z,8,0,5.000000,57
0,EnxrHicdECg,2022-10-30T20:29:53.581443Z,85,0,5.000000,1020
0,tF5T5zm2Kk4,2022-10-30T20:33:43.201512Z,38,0,5.000000,758
...,...,...,...,...,...,...
0,TQFNi7CAb2Q,2022-10-31T00:05:31.6683242Z,9,0,5.000000,2563
0,9DzUGseMs2c,2022-10-31T00:05:32.8680935Z,5,0,5.000000,2850
0,ORDXcEX7xi8,2022-05-29T04:44:37.113576Z,11,0,5.000000,11490
0,2fxhnwxRmN4,2022-10-31T00:05:34.9937874Z,2,0,5.000000,2055


In [56]:
data_info_videos = data_videos.merge(data_likes, how = 'left', left_on = 'id_videos', right_on='id').drop('id_videos', axis = 1)
data_info_videos.to_excel('Data info videos.xlsx', index=False)
data_info_videos

,names_videos,info_videos,duracion_video,page_links,id,dateCreated,likes,dislikes,rating,viewCount
0,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...,5 días 8 minutos y 50 segundos 784 visitas,8 minutos y 50 segundos,https://www.youtube.com/watch?v=cZUmPLANTy0,cZUmPLANTy0,2022-10-30T23:52:23.41786Z,63,0,5.000000,784
1,💚 TODO sobre la herramienta CURVA en CorelDRAW...,"2 semanas 14 minutos y 11 segundos 2,127 visitas",14 minutos y 11 segundos,https://www.youtube.com/watch?v=qZDnqNKSp1U,qZDnqNKSp1U,2022-10-30T20:30:47.357809Z,144,1,4.972414,2117
2,🏃🏼‍♂️ Speedrun LOGO CANGREJO #shorts ▶ DelcaVi...,3 semanas 1 minuto y 5 segundos 368 visitas,1 minuto y 5 segundos,https://www.youtube.com/watch?v=q3FV1NWUVG0,q3FV1NWUVG0,2022-10-08T16:50:03.740318Z,8,0,5.000000,57
3,⭐ Haz tus DISEÑOS con PROPORCIÓN ÁUERA ▶ Tutor...,"3 semanas 10 minutos y 55 segundos 1,028 visitas",10 minutos y 55 segundos,https://www.youtube.com/watch?v=EnxrHicdECg,EnxrHicdECg,2022-10-30T20:29:53.581443Z,85,0,5.000000,1020
4,💚 La herramienta ZOOM Explicada ▶ TUTORIAL Cor...,1 mes 7 minutos y 24 segundos 759 visitas,7 minutos y 24 segundos,https://www.youtube.com/watch?v=tF5T5zm2Kk4,tF5T5zm2Kk4,2022-10-30T20:33:43.201512Z,38,0,5.000000,758
...,...,...,...,...,...,...,...,...,...,...
528,El baño del gato de DelcaVideography,"14 años 7 minutos y 6 segundos 2,563 visitas",7 minutos y 6 segundos,https://www.youtube.com/watch?v=TQFNi7CAb2Q,TQFNi7CAb2Q,2022-10-31T00:05:31.6683242Z,9,0,5.000000,2563
529,"Bancolombia, Medellín, Building, rascasuelos, ...","14 años 1 minuto y 15 segundos 2,850 visitas",1 minuto y 15 segundos,https://www.youtube.com/watch?v=9DzUGseMs2c,9DzUGseMs2c,2022-10-31T00:05:32.8680935Z,5,0,5.000000,2850
530,Edificio Bancolombia Medellín de DelcaVideography,"14 años 28 segundos 11,647 visitas",28 segundos,https://www.youtube.com/watch?v=ORDXcEX7xi8,ORDXcEX7xi8,2022-05-29T04:44:37.113576Z,11,0,5.000000,11490
531,DUCON sistemas modulares para oficina de Delca...,"14 años 5 minutos y 2 segundos 2,055 visitas",5 minutos y 2 segundos,https://www.youtube.com/watch?v=2fxhnwxRmN4,2fxhnwxRmN4,2022-10-31T00:05:34.9937874Z,2,0,5.000000,2055


# Comentarios

In [45]:
data_videos = pd.read_excel('Data info videos.xlsx')
data_videos.head()

,names_videos,info_videos,duracion_video,page_links,id,dateCreated,likes,dislikes,rating,viewCount
0,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...,5 días 8 minutos y 50 segundos 784,8 minutos y 50 segundos,https://www.youtube.com/watch?v=cZUmPLANTy0,cZUmPLANTy0,2022-10-30T23:52:23.41786Z,63,0,5.000000,784
1,💚 TODO sobre la herramienta CURVA en CorelDRAW...,"2 semanas 14 minutos y 11 segundos 2,127",14 minutos y 11 segundos,https://www.youtube.com/watch?v=qZDnqNKSp1U,qZDnqNKSp1U,2022-10-30T20:30:47.357809Z,144,1,4.972414,2117
2,🏃🏼‍♂️ Speedrun LOGO CANGREJO #shorts ▶ DelcaVi...,3 semanas 1 minuto y 5 segundos 368,1 minuto y 5 segundos,https://www.youtube.com/watch?v=q3FV1NWUVG0,q3FV1NWUVG0,2022-10-08T16:50:03.740318Z,8,0,5.000000,57
3,⭐ Haz tus DISEÑOS con PROPORCIÓN ÁUERA ▶ Tutor...,"3 semanas 10 minutos y 55 segundos 1,028",10 minutos y 55 segundos,https://www.youtube.com/watch?v=EnxrHicdECg,EnxrHicdECg,2022-10-30T20:29:53.581443Z,85,0,5.000000,1020
4,💚 La herramienta ZOOM Explicada ▶ TUTORIAL Cor...,1 mes 7 minutos y 24 segundos 759,7 minutos y 24 segundos,https://www.youtube.com/watch?v=tF5T5zm2Kk4,tF5T5zm2Kk4,2022-10-30T20:33:43.201512Z,38,0,5.000000,758


In [5]:
# Reset kernel antes de ejecutar

data_comentarios = []

lista_videos = list(data_videos['page_links'][:150])
nombre_video = list(data_videos['names_videos'][:150])
id_video = list(data_videos['id'][:150])

for video, nombre, id in zip(lista_videos, nombre_video, id_video):

    youtube.open(video)
    response=youtube.video_comments()
    data=response['body']

    df = pd.DataFrame(data)
    df.drop([i for i in ['UserLink', 'user'] if i in df.columns], axis=1, inplace = True)
    df['nombre_video'] = nombre
    df['id'] = id
    
    data_comentarios.append(df)
    sleep(0.5)

data_comentarios_full = pd.concat(data_comentarios)
data_comentarios_full.to_excel('Data comentarios.xlsx', index = False)
data_comentarios_full

Progress: 100%|██████████| 200.0/200 [00:30<00:00,  6.56it/s]
Progress: 100%|██████████| 200.0/200 [00:30<00:00,  6.52it/s]
Progress: 100%|██████████| 200.0/200 [00:14<00:00, 14.01it/s]
Progress: 100%|██████████| 200.0/200 [00:32<00:00,  6.13it/s]
Progress:  60%|██████    | 120.2/200 [00:30<00:21,  3.79it/s]

,Comment,Likes,Time,nombre_video
0,¡Suscríbete y sigue aprendiendo!► https://bit....,1,hace 4 días,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...
1,Las mejores clases sin duda️,1,hace 4 días,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...
2,Muchas gracias por los tutoriales,1,hace 4 días,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...
3,Gracias por compartir,1,hace 4 días,💚Herramienta DIBUJO en Corel DRAW ▶ TUTORIAL |...
0,¡Suscríbete y sigue aprendiendo!► https://bit....,1,hace 4 días,💚 TODO sobre la herramienta CURVA en CorelDRAW...
1,"Excelente, Gracias por compartir tus conocimie...",1,hace 2 semanas,💚 TODO sobre la herramienta CURVA en CorelDRAW...
2,"Excelente, muchas gracias",2,hace 2 semanas,💚 TODO sobre la herramienta CURVA en CorelDRAW...
3,"Estupendo, Muchas Gracias",1,hace 2 semanas,💚 TODO sobre la herramienta CURVA en CorelDRAW...
4,impecable y muy muy claro un abrazo desde URUGUAY,1,hace 2 semanas,💚 TODO sobre la herramienta CURVA en CorelDRAW...
0,¡Suscríbete y sigue aprendiendo!► https://bit....,1,hace 3 semanas,⭐ Haz tus DISEÑOS con PROPORCIÓN ÁUERA ▶ Tutor...


Progress: 100%|██████████| 200.0/200 [00:34<00:00,  5.78it/s]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7377d29e-ada8-4781-9a6b-60a54a1f06a0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>